In [112]:
from tika import parser
import re
import pandas as pd

In [113]:
parsed = parser.from_file("https://www.nyc.gov/assets/dhs/downloads/pdf/dailyreport.pdf")
content = parsed['content'].strip()
date = re.findall(r" (\w+ \d{2}, \d{4})\)", content)[0]
print(date)

2023-12-04 18:12:59,433 [MainThread  ] [INFO ]  Retrieving https://www.nyc.gov/assets/dhs/downloads/pdf/dailyreport.pdf to /var/folders/21/d_7v4w8j3jg95zjx8lfqpw800000gn/T/assets-dhs-downloads-pdf-dailyreport.pdf.


December 03, 2023


In [114]:
date = pd.to_datetime(date)
date

Timestamp('2023-12-03 00:00:00')

In [115]:
TOTAL_SHELTER_CENSUS = {'date': [date]}
SINGLE_ADULTS = {'date': [date]}
FAMILIES_WITH_CHILDREN = {'date': [date]}
ADULT_FAMILIES_CENSUS = {'date': [date]}

In [116]:
content = re.split('NA 0', content)[1].strip()
content = re.split('TOTAL SHELTER CENSUS\n\nDAILY REPORT', content)[0].strip()
numbers = re.split(r'[^\d,]+', content)
numbers

['',
 '56,284',
 '33,147',
 '89,431',
 '89,431',
 '16,053',
 '5,077',
 '21,130',
 '19,398',
 '30,361',
 '33,147',
 '63,508',
 '2,245',
 '4,793']

In [117]:
cleaned_number = []
for i in range(len(numbers)):
    if numbers[i] != '':
        if (i>0) and (numbers[i] != numbers[i-1]):
            cleaned_number.append(re.sub(",", "", numbers[i]))
cleaned_number

['56284',
 '33147',
 '89431',
 '16053',
 '5077',
 '21130',
 '19398',
 '30361',
 '33147',
 '63508',
 '2245',
 '4793']

In [118]:
if len(cleaned_number) == 12:
    TOTAL_SHELTER_CENSUS['Adults'] = [cleaned_number[0]]
    TOTAL_SHELTER_CENSUS['Children'] = [cleaned_number[1]]
    TOTAL_SHELTER_CENSUS['Total Individuals'] = [cleaned_number[2]]
    SINGLE_ADULTS['Men'] = [cleaned_number[3]]
    SINGLE_ADULTS['Women'] = [cleaned_number[4]]
    SINGLE_ADULTS['Total Single Adults'] = [cleaned_number[5]]
    FAMILIES_WITH_CHILDREN['Families'] = [cleaned_number[6]]
    FAMILIES_WITH_CHILDREN['Adults'] = [cleaned_number[7]]
    FAMILIES_WITH_CHILDREN['Children'] = [cleaned_number[8]]
    FAMILIES_WITH_CHILDREN['Individuals'] = [cleaned_number[9]]
    ADULT_FAMILIES_CENSUS['Families'] = [cleaned_number[10]]
    ADULT_FAMILIES_CENSUS['Individuals (Adults)'] = [cleaned_number[11]]
else:
    print('error: lines can not match. Examine the scrapping part')

In [119]:
df1 = pd.DataFrame(TOTAL_SHELTER_CENSUS)
df2 = pd.DataFrame(SINGLE_ADULTS)
df3 = pd.DataFrame(FAMILIES_WITH_CHILDREN)
df4 = pd.DataFrame(ADULT_FAMILIES_CENSUS)

In [120]:
df01 = pd.read_csv("TOTAL_SHELTER_CENSUS.csv")
df02 = pd.read_csv("SINGLE_ADULTS.csv")
df03 = pd.read_csv("FAMILIES_WITH_CHILDREN.csv")
df04 = pd.read_csv("ADULT_FAMILIES_CENSUS.csv")
if date not in df01['date'].values:
    pd.concat([df01, df1]).reset_index(drop=True).to_csv("TOTAL_SHELTER_CENSUS.csv", index=False)
    pd.concat([df02, df2]).reset_index(drop=True).to_csv("SINGLE_ADULTS.csv", index=False)
    pd.concat([df03, df3]).reset_index(drop=True).to_csv("FAMILIES_WITH_CHILDREN.csv", index=False)
    pd.concat([df04, df4]).reset_index(drop=True).to_csv("ADULT_FAMILIES_CENSUS.csv", index=False)